In [86]:
import pandas as pd
import os
from pymongo import MongoClient

In [87]:


def file_route(directory):
    
    df_list = []
    
   
    try:
        files = os.listdir(directory)
    except Exception as e:
        print(f"디렉토리 접근 중 오류 발생: {e}")
        return None
    
    
    for file in files:
        if '주식거래 수수료' in file:
            file_path = os.path.join(directory, file)
            try:
                
                df = pd.read_excel(file_path, header=2)  
                
                df.dropna(axis=0, inplace=True)
                 
                
                df_list.append(df)
                print(f"{file_path} 파일 읽기 성공")
            except Exception as e:
                print(f"{file_path} 파일을 읽는 중 오류 발생: {e}")
    
    # 데이터프레임 병합
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        
        return merged_df
    else:
        print("읽은 파일이 없습니다.")
        return None


# 디렉토리 경로 (예시)
directory = '../datasets/'

# 함수 호출하여 병합된 데이터프레임 반환
merged_data = file_route(directory)

# 병합된 데이터 출력 (있으면)
if merged_data is not None:
    print(merged_data.head())



../datasets/주식거래 수수료_10.xls 파일 읽기 성공
../datasets/주식거래 수수료_1.xls 파일 읽기 성공
../datasets/주식거래 수수료_20241217 .xls 파일 읽기 성공
../datasets/주식거래 수수료_1000.xls 파일 읽기 성공
../datasets/주식거래 수수료_50.xls 파일 읽기 성공
../datasets/주식거래 수수료_100.xls 파일 읽기 성공
../datasets/주식거래 수수료_500.xls 파일 읽기 성공
     회사명             수수료\n부과기준        기준일자  거래금액   구분   오프라인     HTS    ARS  \
0   SK증권  /img/sub/ico_xls.gif  2011/11/18  10만원  변경전  500.0  1150.0  250.0   
1   SK증권  /img/sub/ico_xls.gif  2015/06/01  10만원  변경후  500.0  1150.0  250.0   
2  유안타증권  /img/sub/ico_pdf.gif  2020/09/14  10만원  변경전  500.0   350.0  197.0   
3   대신증권  /img/sub/ico_pdf.gif  2020/09/15  10만원  변경전  494.0   864.0  194.0   
4   교보증권  /img/sub/ico_pdf.gif  2013/12/19  10만원  변경전  490.0   640.0  240.0   

     스마트폰  오프라인.1  HTS.1  ARS.1  스마트폰.1  \
0    50.0   500.0   15.0  250.0    50.0   
1  1150.0   500.0   15.0  250.0  1150.0   
2   100.0   500.0   14.0  197.0   100.0   
3   194.0   494.0  864.0  194.0   194.0   
4   640.0   490.0  640.0  240.0    10.0  

In [88]:

# # 데이터 병합
# merged_df = pd.concat(merged_data, ignore_index=True)

In [89]:

# 병합된 데이터 CSV로 저장
merged_data.to_csv('merged_data3.csv', index=False)


In [ ]:


client = MongoClient('python_jupyternote_mongo-mongodb-1')  
db = client['pandas_db']  
collection = db['stocktradingfees_merge']  


data_dict = merged_data.to_dict(orient='records')   #행(row)별로 리스트로 변환
collection.insert_many(data_dict)  


InsertManyResult([ObjectId('676132f6b128bcafdae29eb0'), ObjectId('676132f6b128bcafdae29eb1'), ObjectId('676132f6b128bcafdae29eb2'), ObjectId('676132f6b128bcafdae29eb3'), ObjectId('676132f6b128bcafdae29eb4'), ObjectId('676132f6b128bcafdae29eb5'), ObjectId('676132f6b128bcafdae29eb6'), ObjectId('676132f6b128bcafdae29eb7'), ObjectId('676132f6b128bcafdae29eb8'), ObjectId('676132f6b128bcafdae29eb9'), ObjectId('676132f6b128bcafdae29eba'), ObjectId('676132f6b128bcafdae29ebb'), ObjectId('676132f6b128bcafdae29ebc'), ObjectId('676132f6b128bcafdae29ebd'), ObjectId('676132f6b128bcafdae29ebe'), ObjectId('676132f6b128bcafdae29ebf'), ObjectId('676132f6b128bcafdae29ec0'), ObjectId('676132f6b128bcafdae29ec1'), ObjectId('676132f6b128bcafdae29ec2'), ObjectId('676132f6b128bcafdae29ec3'), ObjectId('676132f6b128bcafdae29ec4'), ObjectId('676132f6b128bcafdae29ec5'), ObjectId('676132f6b128bcafdae29ec6'), ObjectId('676132f6b128bcafdae29ec7'), ObjectId('676132f6b128bcafdae29ec8'), ObjectId('676132f6b128bcafdae29e

In [91]:
query = {'구분': '변경후'}
cursor = collection.find(query)

In [92]:
# 데이터를 DataFrame으로 변환
data = pd.DataFrame(list(cursor))

In [93]:
# '스마트폰.1' 컬럼에 대한 평균 계산
smartphone_avg = data['스마트폰.1'].mean()
smartphone_avg

16501.77055449331

In [94]:
# 스마트폰 수수료 평균 이상인 증권사들만 필터링
df_filtered = data[data['스마트폰.1'] >= smartphone_avg]
df_filtered

,_id,회사명,수수료\n부과기준,기준일자,거래금액,구분,오프라인,HTS,ARS,스마트폰,오프라인.1,HTS.1,ARS.1,스마트폰.1,비고
1,67611df0b140033f893c1fb9,SK증권,/img/sub/ico_xls.gif,2015/06/01,1억원,변경후,500000.0,120000.0,250000.0,120000.0,500000.0,15000.0,250000.0,120000.0,증권통App 수수료율 변경
2,67611df0b140033f893c1fbc,유안타증권,/img/sub/ico_pdf.gif,2021/01/04,1억원,변경후,500000.0,65000.0,197000.0,100000.0,500000.0,14000.0,197000.0,100000.0,NaN
6,67611df0b140033f893c1fc7,상상인증권,/img/sub/ico_pdf.gif,2020/04/23,1억원,변경후,490000.0,140000.0,200000.0,140000.0,490000.0,140000.0,200000.0,140000.0,NaN
7,67611df0b140033f893c1fc9,대신증권,/img/sub/ico_pdf.gif,2021/01/01,1억원,변경후,472296.0,97296.0,197296.0,197296.0,472296.0,97296.0,197296.0,197296.0,NaN
8,67611df0b140033f893c1fcc,교보증권,/img/sub/ico_pdf.gif,2021/09/01,1억원,변경후,499200.0,99200.0,249200.0,149200.0,499200.0,99200.0,249200.0,149200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,676132d5b128bcafdae29e8c,NH투자증권,/img/sub/ico_pdf.gif,2020/06/10,1000만원,변경후,49720.0,14916.0,19716.0,19716.0,49720.0,14916.0,19716.0,19716.0,NAMUH 1000
1436,676132f6b128bcafdae29ebb,SK증권,/img/sub/ico_xls.gif,2015/06/01,1억원,변경후,500000.0,120000.0,250000.0,120000.0,500000.0,15000.0,250000.0,120000.0,증권통App 수수료율 변경
1437,676132f6b128bcafdae29ebf,NH투자증권,/img/sub/ico_pdf.gif,2020/06/10,1억원,변경후,497200.0,96664.0,197164.0,197164.0,497200.0,96664.0,197164.0,197164.0,NAMUH 10000
1438,676132f6b128bcafdae29ec1,아이엠증권,/img/sub/ico_hwp.gif,2012/05/02,1억원,변경후,497200.0,97200.0,247200.0,97200.0,497200.0,97200.0,247200.0,97200.0,유관수수료인하


In [95]:
#증권사별 스마트폰 수수료 평균 
df_grouped = df_filtered.groupby('회사명')['스마트폰.1'].mean().reset_index()
df_grouped

,회사명,스마트폰.1
0,DB금융투자,99164.0
1,IBK투자증권,100000.0
2,KB증권,120000.0
3,NH투자증권,108440.0
4,SK증권,120000.0
5,교보증권,149200.0
6,대신증권,108513.0
7,메리츠증권,88132.0
8,부국증권,150000.0
9,비엔케이투자증권,150000.0


In [96]:
# 전체 스마트폰 수수료 평균
overall_avg = smartphone_avg
overall_avg

16501.77055449331

In [97]:

df_grouped.to_csv('smartphone_fee_avgup3.csv', index=False)